In [ ]:
import numpy as np
#cv2 a.k.a. opencv is a package for dealing with images
import cv2 as cv
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import math as mh
from IPython.display import clear_output

# Getting and preprocessing data

In [ ]:
#LOAD IMAGES
# chose this size so that it approximates images well but is also highly divisible by 2
# because some of the neural network layers shrink the size by factor of 2

XDIM = 300 #2816
YDIM = 300 #11*XDIM//8 

def load_marsh_images(folder):
    images = []
    
    for i in range(1,12):
        # reading PIE
        #img = cv.imread(os.path.join(folder,"LC08_L1TP_012030_20130404_20170310_01_T1_B%d_subset.png" % i))
        # reading VCR
        img = cv.imread(os.path.join(folder,"LC08_L1TP_014034_20130516_20170310_01_T1_B%d_subset.png" % i))
        # reading GCE
        #img = cv.imread(os.path.join(folder,"LC08_L1TP_016038_20130328_20170310_01_T1_B%d_subset.png" % i))
        if img is not None:
            # resize our higher resolution band 8 to be the same with others
            if (i == 8):
                img = cv.resize(img,(xDIM, yDIM))
            else:
                yDIM = img.shape[0]
                xDIM = img.shape[1]
            #padding/resizing them with white so they are the same size
            img = cv.copyMakeBorder(img,mh.ceil((YDIM-yDIM)/2),mh.floor((YDIM-yDIM)/2),mh.ceil((XDIM-xDIM)/2),mh.floor((XDIM-xDIM)/2),cv.BORDER_CONSTANT, None, value = 0)
            #img = cv.resize(img,(XDIM, YDIM))
            #convert to black and white
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            #subset images to zoom in
            img = img #img[199:499,199:399] for VCR
            #warp so the mask is square
            #img = cv.warpPerspective(img,Mpad,(SIZE+2*PAD,SIZE+2*PAD))
            images.append(img)
    return images

#get images with PIE thrown out

#gce_imgs = load_marsh_images("train_jpg/GCE")
#pie_imgs = load_marsh_images("train_jpg/PIE")
#vcr_imgs = load_marsh_images("train_jpg/VCR")

#gce_imgs = load_marsh_images("train_jpg/Jul2023/GCE")
#pie_imgs = load_marsh_images("train_jpg/Jul2023/PIE")
vcr_imgs = load_marsh_images("train_jpg/Jul2023/VCR")

images = np.array([
    #gce_imgs
    #pie_imgs
    vcr_imgs
    ])
NUM_SITE = images.shape[0]
#YDIM=images.shape[2]
#XDIM=images.shape[3]
#XDIM=images[site_num,5].shape[0]
#YDIM=images[site_num,5].shape[1]

In [ ]:
# view one of our beautiful images
site_num=0 #selecting site to train
plt.imshow(images[site_num,5])
print(images[site_num,5].shape)

In [ ]:
#GET MASKS


#gce_mask = cv.imread("new-masks/updated_GCE_mask.jpg")
#pie_mask = cv.imread("new-masks/updated_PIE_mask.jpg")
#vcr_mask = cv.imread("new-masks/updated_VCR_map.jpg")

#new data

#gce_mask = cv.imread("new-masks/Jul2023/ready for ML GCE1.png")
#gce_mask = cv.resize(gce_mask,(xDIM,yDIM))
#pie_mask = cv.imread("new-masks/Jul2023/ready for ML PIE1.png")
#pie_mask = cv.resize(pie_mask,(xDIM,yDIM))
vcr_mask = cv.imread("new-masks/Jul2023/ready for ML VCR1.png")
#print(vcr_mask.shape)
#vcr_mask = cv.resize(vcr_mask,(xDIM,yDIM))
#subset masks to zoom in
#vcr_mask = vcr_mask[199:499,199:399]

new_masks = [
         #cv.cvtColor(gce_mask, cv.COLOR_BGR2GRAY) 
         #cv.cvtColor(pie_mask, cv.COLOR_BGR2GRAY) 
         cv.cvtColor(vcr_mask, cv.COLOR_BGR2GRAY)
            ]
# exploring data to set thresholds
#plt.imshow(new_masks[0])
#plt.colorbar()
#plt.hist(new_masks[0])
#plt.show

#new_masks = [gce_mask,
#            pie_mask,
#            vcr_mask]

masks = []
for m in new_masks:
    #m = cv.resize(m,(YDIM,XDIM))
    # padding the masks in the same way with imgs so they match sizes
    yDIM = m.shape[0]
    xDIM = m.shape[1]    
    m = cv.copyMakeBorder(m,mh.ceil((YDIM-yDIM)/2),mh.floor((YDIM-yDIM)/2),mh.ceil((XDIM-xDIM)/2),mh.floor((XDIM-xDIM)/2),cv.BORDER_CONSTANT, None, value = 0)
    masks.append(m)
new_masks = masks
print(new_masks[0].shape)
#YDIM = new_masks[0].shape[0]
#XDIM = new_masks[0].shape[1]

In [ ]:
#PREPROCESS MASKS
#change the values in the mask to integers from 1 to 5 instead of the weird 
# 0-255 values currently in there. Ideally the GIS software would output them like 
# this already, but c'est la vie.

#hist, bins = np.histogram(new_masks[site_num], bins=85)

#print (hist)
#print (bins)

THRESH = [10,26,40,60,76] #found by exploring the data
# VCR        [10,          26,     40,      60,      76]
#    unlabeled   tidal_flat  upland    pond    channel  marsh
# GCE        [10,      20,         46,      60]
#    unlabeled   marsh    tidal_flat  upland  channel
# PIE        [12,      20,         46,      55]
#    unlabeled   marsh    ponds      upland  channel 
new_mask = np.array(new_masks[0])
unlabeled = (new_mask <= THRESH[0])
tidal_flat = np.logical_and(THRESH[0] < new_mask, new_mask <= THRESH[1])
upland = np.logical_and(THRESH[1] < new_mask, new_mask <= THRESH[2])
pond = np.logical_and(THRESH[2] < new_mask, new_mask <= THRESH[3])
channel = np.logical_and(THRESH[3] < new_mask, new_mask <= THRESH[4])
marsh = THRESH[4] < new_mask

#to throw away tidal_flat
unlabeled = np.logical_or(tidal_flat,unlabeled)
#to throw away pond
unlabeled = np.logical_or(pond,unlabeled)
masks = [channel,marsh,upland,unlabeled]

int_mask = np.zeros_like(new_mask,dtype=int)
for i,m in enumerate(masks):
    int_mask = int_mask + i*m.astype(int)
masks = np.reshape(int_mask, (NUM_SITE,1,YDIM,XDIM))

#get frequencies of each class
ind,counts = np.unique(masks,return_counts=True)

In [ ]:
# use the above image to set the marsh colors:

class_dict = {0: 'water', 1: 'marsh', 2: 'upland', 3: 'unlabeled'}

In [ ]:
# view one of our beautiful masks
cmap = plt.cm.get_cmap('viridis', 4)
plt.imshow(masks[site_num,0,:,:],vmax=3,vmin=0,cmap=cmap)
cbar = plt.colorbar(ticks=[3/8 + i*(3.0/4.0) for i in range(4)],orientation='horizontal')
cbar.set_ticklabels([class_dict[i] for i in range(len(class_dict))])

In [ ]:
np.save('numpymasks',masks)

In [ ]:
np.shape(masks)

In [ ]:
#TEST AND TRAIN SPLITS
# we need to hide part of each image from the neural net while training 
# so that we can later use the hidden part for testing.
# I do a naive thing where I just convert the upper 5/8 of the mask to the unknown class.
# I also convert to tensor (data format for tensorflow)

#Yiyang
# It seems that tensor requires a rectangular data format where we simply don't have due to mismatch dimensions from 3 sites.
# we could padd the images or use something like tensor torch?

not_care = np.max(masks)

te_masks = np.zeros_like(masks)
tr_masks = np.zeros_like(masks)

te_masks.fill(not_care)
tr_masks.fill(not_care)

te_masks[:,0,:3*XDIM//8,:]=masks[:,0,:3*XDIM//8,:]
tr_masks[:,0,3*XDIM//8:,:]=masks[:,0,3*XDIM//8:,:]


def get_tensor(images):
    images_tensor = tf.convert_to_tensor(images,dtype =tf.float32)
    SAMPLES,BANDS,HEIGHT,WIDTH = images_tensor.shape
    image_tensor = tf.transpose(images_tensor,[0,2,3,1])
    return image_tensor

train_images = get_tensor(images)

train_masks = get_tensor(tr_masks)

test_images = get_tensor(images)

test_masks = get_tensor(te_masks)


In [ ]:
# view one of our testing masks
plt.imshow(te_masks[site_num,0])

In [ ]:
#CREATE TENSORFLOW DATASET AND AUGMENT DATA
# datasets are a data format that makes training fast
# each augmentation transformation
#changes image and mask in exactly the same way

train_dataset = tf.data.Dataset.from_tensor_slices((train_images,train_masks))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images,test_masks))


class Augment(tf.keras.layers.Layer):
  def __init__(self, seed=42):
    super().__init__()
    # both use the same seed, so they'll make the same random changes.
    self.augment_inputs = tf.keras.Sequential([
      tf.keras.layers.RandomFlip("horizontal_and_vertical",seed=seed),
      tf.keras.layers.RandomRotation(1.0,seed=seed),
      tf.keras.layers.RandomZoom(.5, .5,seed =seed)
    ])
    self.augment_labels = tf.keras.Sequential([
      tf.keras.layers.RandomFlip("horizontal_and_vertical",seed=seed),
      tf.keras.layers.RandomRotation(1.0,seed=seed),
      tf.keras.layers.RandomZoom(.5, .5,seed =seed)
    ])
    
  def call(self, inputs, labels):
    inputs = self.augment_inputs(inputs)
    labels = self.augment_labels(labels)
    return inputs, labels

with tf.device('/cpu:0'):
    train_batches = (
    train_dataset
    .cache()
    .shuffle(NUM_SITE)
    .batch(NUM_SITE)
    #.repeat()
    #.map(Augment()) ## not augmenting for linreg!!
    .prefetch(buffer_size=tf.data.AUTOTUNE))

test_batches = test_dataset.batch(3)

# Defining our model

In [ ]:
#linear regression model - no hidden layer

def shallow_model(output_channels:int):
    inputs = tf.keras.layers.Input(shape=[None, None, 11])
    
    #down
    x = tf.keras.layers.Conv2D(filters=output_channels,kernel_size=1,strides=1)(inputs)
    
    #x = last(x)
    
    return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
# linear regression on small neighborhood a la George

def shallow_model(output_channels:int):
    inputs = tf.keras.layers.Input(shape=[None, None, 11])
    
    #down
    x = tf.keras.layers.Conv2D(filters=output_channels,kernel_size=3,strides=1,padding='same')(inputs)
    
    #x = last(x)
    
    return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
# DEFINE MODEL
# starting at inputs, each line is applied sequentially. 
# the output of each layer is an image (well, it may have many more than 3 bands)
# which will be fed into the next layer.
# finally the output will be an image with 4 (number of classes) bands 
# with the intensity at pixel p in class c reflecting the network's confidence that 
# that pixel p belongs to class c. 
# the mask is then created by assigning p to the class which has highest confidence.

#best one so far

def shallow_model(output_channels:int):
    inputs = tf.keras.layers.Input(shape=[None, None, 11])
    
    #down
    x = tf.keras.layers.Conv2D(filters=5,kernel_size=2,strides=2,activation="relu",padding='same')(inputs)
    
    #maxpool
    #x = tf.keras.layers.MaxPool2D(pool_size=(2, 2),padding='same')(x)
    
    x = tf.keras.layers.Conv2D(5,2,2,activation="relu",padding='same')(x)
    
    #maxpool
    #x = tf.keras.layers.MaxPool2D(pool_size=(2, 2),padding='same')(x)
    
    #up
    x = tf.keras.layers.Conv2DTranspose(5,2,2,activation="relu",padding='same')(x)
    
    last = tf.keras.layers.Conv2DTranspose(
    filters=5, kernel_size=2, strides=2,activation="relu",
    padding='same')
    
    x = last(x)
    
    x = tf.keras.layers.Conv2D(output_channels,2,1,activation="relu",padding='same')(x)
    #x = tf.keras.layers.MaxPool2D(pool_size=(2, 2),padding='same')(x)

    
    return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
# COMPILE THE MODEL

#4 classes when tidal flats are unlabeled 

OUTPUT_CLASSES = 4
#OUTPUT_CLASSES=5


model = shallow_model(output_channels=OUTPUT_CLASSES)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'],
              weighted_metrics=['accuracy']
             )

In [ ]:
# look at architecture

tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# summarize model

model.summary()

# Viewing model output

In [ ]:
#VIEWING MODEL OUTPUT
# we should probably put legends for the classes, 
# otherwise its easy to get confused between colors

for image,mask in test_dataset.take(2):
    sample_image,sample_mask = image, mask
for image,mask in train_dataset.take(2):
    sample_train_image,sample_train_mask = image, mask

def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image, mask[0,0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

def show_train_predictions(dataset=None, num=2):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0,0], create_mask(pred_mask)])
  else:
    display([sample_train_image, sample_train_mask,
             create_mask(model.predict(sample_train_image[tf.newaxis, ...]))])
    
def display(display_list):
  plt.figure(figsize=(20, 20))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1,len(display_list),i+1)
    plt.title(title[i])
    if display_list[i].shape[-1]==1:
        mi = 0
        ma = 3
    else:
        mi = 4
        ma = 7
    plt.imshow(tf.keras.utils.array_to_img(display_list[i][:,:,mi:ma]),vmin=0,vmax=255)
    plt.axis('off')
  plt.show()
# print(pred_mask_int.shape)
# print(pred_mask_int[0,90,180,0].numpy())

In [ ]:
# check what our network outputs now
# if training hasn't happened should look totally random
show_train_predictions()

# Saving and loading

In [ ]:
# SAVING AND LOADING
# be very careful to choose the filename so as not to overwrite a good model that 
# took hours or days to train!

#saving
model.save('saved-models/VCR_AGU23')

In [ ]:
# loading
model = tf.keras.models.load_model('saved-models/Bmodel-5000Oct24')

# Training

In [ ]:
#TRAINING MODEL

#callback is just to do display info during training
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if epoch % 5==0:
        clear_output(wait=True)
        #show_predictions()
        show_train_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))
    
# be careful to send your logs to a useful filename, right now it's /logs/Demo

tb_callback = tf.keras.callbacks.TensorBoard('./logs/test', update_freq=5)

## adding sample weights so that the final class is ignored 

def add_sample_weights(image, label):
  # The weights for each class, with the constraint that:
  #     sum(class_weights) == 1.0
  class_weights = tf.constant([1.5/counts[0],1.5/counts[1],1/counts[2], 0.0])
  #class_weights = tf.constant(inv_freq)
  class_weights = class_weights/tf.reduce_sum(tf.abs(class_weights))

  # Create an image of `sample_weights` by using the label at each pixel as an 
  # index into the `class weights` .
  sample_weights = tf.gather(class_weights, indices=tf.cast(label, tf.int32))

  return image, label, sample_weights

#actually training 

EPOCHS = 1000
# 10 epochs for demo 
#change to 3000 or so to retrain for real

model_history = model.fit(train_batches.map(add_sample_weights), 
                          #class_weight = {0:1,1:1,2:1,3:1,4:0},
                          epochs=EPOCHS,
                          #steps_per_epoch=STEPS_PER_EPOCH,
                          #validation_steps=VALIDATION_STEPS,
                          validation_data=test_batches.map(add_sample_weights),
                          callbacks=[DisplayCallback(),tb_callback]
                         )

# logging

In [ ]:
# change to reload_ext
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/test

# Analyzing the predictions (confusion matrix)

In [ ]:
import seaborn as sns
COLOR='red'
plt.rcParams['text.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['xtick.color'] = COLOR
plt.rcParams['ytick.color'] = COLOR

In [ ]:
test_masks_int = tf.cast(test_masks, dtype=tf.int32)
class_list = list(class_dict.values())
class_list.remove('unlabeled')

In [ ]:
def create_mask_full(pred_mask):
     pred_mask = tf.argmax(pred_mask, axis=-1)
     pred_mask = pred_mask[..., tf.newaxis]
     return pred_mask

In [ ]:
pred_mask = model.predict(test_images)
pred_mask.shape
pred_mask_int=create_mask_full(pred_mask)

In [ ]:
confusion_mtx = tf.math.confusion_matrix(tf.reshape(test_masks_int, [-1]), tf.reshape(pred_mask_int, [-1]))[:3,:3]

In [ ]:
row_sums = np.sum(confusion_mtx, axis=1)
col_sums = np.sum(confusion_mtx, axis=0)
confusion_mtx_user = np.diag(1/row_sums) @ confusion_mtx.numpy()
confusion_mtx_pdsr = confusion_mtx.numpy() @ np.diag(1/col_sums)

In [ ]:
# unnormalized
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, xticklabels=class_list, yticklabels=class_list, 
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Truth')
plt.show()

In [ ]:
#normalized truth
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx_pdsr, xticklabels=class_list, yticklabels=class_list, 
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Truth')
plt.show()